In [4]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
import pyspark
from pyspark.sql import SparkSession

In [8]:
spark = SparkSession.builder.appName('Movie Recommendation').getOrCreate()

from pyspark.mllib.recommendation import Rating
from pyspark.mllib.recommendation import ALS

raw = spark.sparkContext.textFile("/content/ratings.dat")

mydata = [(2, 0.01)]

mydatardd = spark.sparkContext.parallelize(mydata).map(lambda x: Rating(0, x[0], x[1]))

def parseRating(str):
    fields = str.split("::")
    assert(len(fields) == 4)
    return Rating(int(fields[0]), int(fields[1]), float(fields[2]))

ratings = raw.map(parseRating)
totalratings = ratings.union(mydatardd)
model = ALS.train(totalratings, rank=8, iterations=5, lambda_=1.0)
products = model.recommendProducts(1, 10)

products

Py4JJavaError: ignored

In [9]:
from pyspark.mllib.linalg import Vectors
from pyspark.mllib.stat import Statistics

vectorRDD = totalratings.map(lambda rating: Vectors.dense(rating.user, rating.product, rating.rating))
summary = Statistics.colStats(vectorRDD)

print(summary.mean())
print(summary.variance())
print(summary.numNonzeros())

Py4JJavaError: ignored